#Procesamiento de Lenguaje Natural
## Práctica 6: Análisis de Opinión
* Gónzalez Chacón Mónica
* López Salazar Esmeralda Leticia
* Rodriguez Nuñez Diego Eduardo

In [2]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 27.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
!pip install imbalanced-learn

In [4]:
import pandas as pd
import re
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [6]:
datos = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Sexto Semestre/Natural Language Processing/SentimentAnalysis/Rest_Mex_2022.xlsx")

## Diccionario de datos

|Número de variable|Nombre de la variable|Tipo de dato| Descripción|
|--|---------------------|------------|------------|
|1| Title                   | Categórico        | El título de la opinión.
|2| Opinion                   | Categórico        | Opinión.
|3| Polarity                   | Discreto        | Es un valor númerico que va de 1 a 5, donde el 1 representa una opinión negativa y el 5 es una opinión positiva.
|4| Attraction                   | Categórico        | Corresponde al lugar del cual se escribe la opinión/reseña.

In [7]:
datos.head(2)

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant


In [8]:
datos.shape

(30212, 4)

## Corpus Preprocessing

In [9]:
datos.isnull().sum()

,0
Title,2
Opinion,2
Polarity,0
Attraction,0


In [10]:
datos=datos.dropna()

### División de datos en características y clase

In [ ]:
X = datos["Title"].astype(str)+ " " + datos["Opinion"].astype(str)

In [12]:
print(X)

0        Pésimo lugar Piensen dos veces antes de ir a e...
1        No vayas a lugar de Eddie Cuatro de nosotros f...
2        Mala relación calidad-precio seguiré corta y s...
3        Minusválido? ¡No te alojes aquí! Al reservar u...
4        Es una porqueria no pierdan su tiempo No pierd...
                               ...                        
30207    Verdadera joya arquitectónica Es una construcc...
30208    Romántico Muy al estilo de Romeo y Julieta es ...
30209    Parece un castillo Ideal para subir las escali...
30210    Imperdible Es imperdible, de ahí puedes ver mu...
30211    Muy bonita vista No te puedes ir de Guanajuato...
Length: 30208, dtype: object


In [13]:
y = datos["Polarity"].values

In [14]:
print(y)

[1 1 1 ... 5 5 5]


### División de los datos en conjunto de prueba y de entrenamiento

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

In [16]:
print(X_train)

22149    Estancia maravillosa Personal y un servicio im...
25643    Cesar de joy squad Excelente servicio el de ce...
21050    Excelente servicio Muy bonita vista fastastica...
21560    Vacaciones en familia Las instalaciones muy li...
19436    El mejor viaje de mi vida! Tenemos una multipr...
                               ...                        
13124    Me encanta este lugar!! Visitamos el santuario...
19650    ¡GUAU! Qué ver llegar temprano y ver las más i...
9846     mejor en Nuevo Nos quedamos aquí por la tercer...
10800    Riquisimo Instalaciones acogedoras comida deli...
2732     Detalle turístico Es un detalle turístico, un ...
Length: 24166, dtype: object


In [17]:
print(y_train)

[5 5 5 ... 5 5 3]


### Normalización de Texto y tratamiento de repetición de caracteres

In [18]:
nlp = spacy.load("es_core_news_sm")

In [19]:
def eliminar_repeticiones(texto):
    # Reemplazar repeticiones excesivas de caracteres con solo una instancia del carácter
    texto_corregido = re.sub(r'(.)\1{1,}', r'\1', texto)  # Reemplaza 2 o más caracteres repetidos por uno solo
    texto_corregido = re.sub(r'\b\w\b', '', texto_corregido)  # Elimina las palabras de solo un carácter

    # Eliminar espacios extras (reemplaza múltiples espacios con un solo espacio)
    texto_corregido = re.sub(r'\s+', ' ', texto_corregido)
    texto_corregido = texto_corregido.strip()
    return texto_corregido

def normalizar_texto_con_reparacion(texto):
    # Paso 1: Limpiar el texto (eliminar caracteres especiales y convertir a minúsculas)
    texto_limpio = ''.join(e for e in texto if e.isalpha() or e.isspace()).lower()

    # Paso 2: Eliminar repeticiones excesivas de caracteres
    texto_corregido = eliminar_repeticiones(texto_limpio)

    # Paso 3: Tokenización y lematización
    doc = nlp(texto_corregido)
    lemas = [token.lemma_ for token in doc if token.is_stop or not token.is_stop]
    return " ".join(lemas)

In [22]:
registros_normalizados = X_train.apply(lambda x: normalizar_texto_con_reparacion(str(x)))
print(registros_normalizados.head(4))

22149    estancia maravilós personal uno servicio impre...
25643    cesar de joy squad excelente servicio el de ce...
21050    excelente servicio mucho bonito vista fastasti...
21560    vacación en familia el instalación mucho limpi...
dtype: object


In [23]:
registros_normalizados[1133]

'momia no no quisierar volver ir ver puro gente muerto en vacación señora con su bebé no saber ni porque ir'

En esta opinión se puede observar que se elimino totalmente "mmmm" porque al momento de reducirlo a un solo caracter quedaba "m" lo cual consideramos que no era de utilidad, por lo que aquellos terminos que solo tenían un caracter fueron eliminados.

In [24]:
print("Opinión sin modificación :D\n",X_train[1133])

Opinión sin modificación :D
 Momias NO Mmmmm no quisiera volver a ir... ¿ver pura gente muerta? ¿En vacaciones? ¿Señoras con sus bebés? No sé ni porque fui.


## Representación de Texto
Elegimos utilizar la binarizada debido a que en nuestros experimentos individuales, nos funcionó mejor esa representación

In [25]:
vectorizer = CountVectorizer(binary=True,max_features=5000)
X_binario = vectorizer.fit_transform(registros_normalizados)
X_binario_array = X_binario.toarray()

In [26]:
print(f"Longitud de X_train_normalizado: {len(X_binario_array)}")

Longitud de X_train_normalizado: 24166


## Balanceo de clases

###Oversampling
Primero probaremos que tal funciona la clasificación haciendo oversampling

In [27]:
y_train_series = pd.Series(y_train)
print(y_train_series.value_counts())

5    16725
4     4714
3     1699
2      585
1      443
Name: count, dtype: int64


Tenemos un gran desbalance de clases, por lo que vamos a balancearlas a continuación

In [28]:
sampling_strategy = {1: 16725, 2: 16725, 3: 16725, 4: 16725, 5: 16725}
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_binario_array, y_train)

print(pd.Series(y_train_balanced).value_counts())

5    16725
4    16725
2    16725
3    16725
1    16725
Name: count, dtype: int64


In [29]:
print(f"Longitud de y_train sin balanceo: {len(y_train)}")
print(f"Longitud de y_train balanceada: {len(y_train_balanced)}")

Longitud de y_train sin balanceo: 24166
Longitud de y_train balanceada: 83625


### Undersampling combinado con Oversampling
Como la clase minoritaria tiene muy pocas muestras, el undersampling que haremos estará ajustado a 3500 muestras.

In [30]:
y_train_series = pd.Series(y_train)
class_counts = y_train_series.value_counts()
sampling_strategy = {class_id: 3500 for class_id, count in class_counts.items() if count > 3500}
undersampler = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_train_undersampled, y_train_undersampled = undersampler.fit_resample(X_binario_array, y_train)


sampling_strategy = {3: 3500, 2: 3500, 1: 3500}
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_train_balanced2, y_train_balanced2 = smote.fit_resample(X_train_undersampled, y_train_undersampled)

print(pd.Series(y_train_balanced).value_counts())

5    16725
4    16725
2    16725
3    16725
1    16725
Name: count, dtype: int64


In [32]:
# ola :D, esta es una ver optimizada pq la otra se avienta mil machotes mas, asi que, aki ta esta :D